#### Seq2Seq 구현 연습.

In [1]:
#Encoder 

import torch 
import torch.nn as nn 
import torch.functional as F 


In [3]:
#Encoder부터..

class Encoder(nn.Module):
    
    #Encoder를 위한 생성자를 먼저 구현해야합니다.
    def __init__(self,input_dimension,embedding_dimension,hidden_dimension,n_layers,dropout):
        super(Encoder,self).__init__()
        self.input_dimension=input_dimension 
        self.embedding_dimension = embedding_dimension 
        self.hidden_dimension = hidden_dimension
        self.n_layers = n_layers 
        self.dropout = dropout 


        #임베딩 입력 값에 대해 Embedding 수행 
        #세로 : 임베딩할 단어의 개수 
        #가로 : 임베딩을 하게 될 길이.
        self.embedding = nn.Embedding(self.input_dimension,self.embedding_dimension)

        self.rnn = nn.LSTM(self.embedding_dimension,self.hidden_dimension,n_layers,dropout=dropout)
        self.dropout=nn.Dropout(dropout)

    def forward(self,x):

        embedded = self.dropout(self.embedding(x))

        #초기 hidden state는 zero tensor
        outputs,(hidden,cell)= self.rnn(embedded)

        return hidden,cell

In [4]:
#decoder 

class Decoder(nn.Module):

    def __init__(self,output_dimension,embedding_dimension,hidden_dimension,n_layers,dropout):
        super(Decoder,self).__init__()

        self.output_dimension = output_dimension
        self.embedding_dimension = embedding_dimension
        self.hidden_dimension = hidden_dimension
        self.n_layers = n_layers 
        self.dropout = dropout 


        #context vector를 입력받아 embedding dimension 출력
        self.embedding = nn.Embedding(num_embeddings=output_dimension,embedding_dim=embedding_dimension)

        #embedding을 입력받아서 hidden state 크기의 hidden state, cell출력 
        self.rnn = nn.LSTM(embedding_dimension,hidden_dimension,dropout=dropout)

        self.fc_out = nn.Linear(self.hidden_dimension,self.output_dimension)

        self.dropout=nn.Dropout(dropout)
    
    def forward(self,input,hidden,cell):

        #input : [batch size]

        #hidden : [n_layers * n_di]

        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))

        output,(hidden,cell)=self.rnn(embedded,(hidden,cell))

        
        prediction = self.fc_out(output.squeeze(0))

        return prediction,hidden,cell



In [5]:
# Seq2Seq
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        # encoder와 decoder의 hid_dim이 일치하지 않는 경우 에러메세지
        assert encoder.hid_dim == decoder.hid_dim, \
            'Hidden dimensions of encoder decoder must be equal'
        # encoder와 decoder의 hid_dim이 일치하지 않는 경우 에러메세지
        assert encoder.n_layers == decoder.n_layers, \
            'Encoder and decoder must have equal number of layers'

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: [src len, batch size]
        # trg: [trg len, batch size]
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0] # 타겟 토큰 길이 얻기
        trg_vocab_size = self.decoder.output_dim # context vector의 차원

        # decoder의 output을 저장하기 위한 tensor
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # initial hidden state
        hidden, cell = self.encoder(src)

        # 첫 번째 입력값 <sos> 토큰
        input = trg[0,:]

        for t in range(1,trg_len): # <eos> 제외하고 trg_len-1 만큼 반복
            output, hidden, cell = self.decoder(input, hidden, cell)

            # prediction 저장
            outputs[t] = output

            # teacher forcing을 사용할지, 말지 결정
            teacher_force = random.random() < teacher_forcing_ratio

            # 가장 높은 확률을 갖은 값 얻기
            top1 = output.argmax(1)

            # teacher forcing의 경우에 다음 lstm에 target token 입력
            input = trg[t] if teacher_force else top1

        return outputs